# Polyglot-ko-12.8b 모델을 KoALPACA 모델로의 학습 후 평가

## 1. 개요
* 모델명 :[meta-llama/Llama-2-7b-hf](https://huggingface.co/meta-llama/Llama-2-7b-hf) 모델을 아래의 QLoRA로 SFT 한 모델
* 데이터셋
    * 한국어 Alpaca Dataset : [ko_alpaca_data.json](https://github.com/Beomi/KoAlpaca/blob/main/ko_alpaca_data.json)
    * 네이버 지식인 베스트 데이터 : [KoAlpaca_v1.1.json](https://raw.githubusercontent.com/Beomi/KoAlpaca/main/KoAlpaca_v1.1.jsonl)

In [1]:
import os
import torch
from torch.utils.data import Dataset

from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel,
)
import transformers
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    GenerationConfig,
    TextStreamer
)

import pandas as pd
import json

## 2. Set Arguments

### 2.1 base 관련 파라미터

In [2]:
CHECKPOINT_DIR = '/workspace/polyglot_koalpaca_finetuning/output/expt-4epochs'

In [3]:
BASE_PATH = '/workspace/polyglot_koalpaca_finetuning'
RANDOM_SEED = 777
HUGGINGFACE_TOKEN = ''

In [4]:
CACHE_DIR=os.path.join('/workspace', ".cache") 

### 2.2 Model 관련 파라미터

In [5]:
MODEL_NAME_OR_PATH = 'beomi/polyglot-ko-12.8b-safetensors'
TORCH_DTYPE=torch.float16

### 2.3 양자화 관련 파라미터

In [6]:
LOAD_IN_4BIT=True                                   # Enable 4-bit quantization
BNB_4BIT_QUANT_TYPE="nf4"                           # BNB 4-bit quantization type
BNB_4BIT_COMPUTE_DTYPE=torch.bfloat16               # BNB 4-bit compute dtype
BNB_4BIT_USE_DOUBLE_QUANT=True                      # BNB 4-bit use double quantization

## 2.4 LoRA 관련 파라미터

In [7]:
R=8                                                 # Lora attention dimension
LORA_ALPHA=16                                       # Lora alpha parameter
LORA_DROPOUT=0.05                                   # Lora dropout probability
FAN_IN_FAN_OUT=False                                # Lora fan in fan out
BIAS="none"                                         # Lora bias type
TARGET_MODULES=["q_proj", "v_proj"]                 # Lora target modules
INFERENCE_MODE=False                                # Inference mode
TASK_TYPE="CAUSAL_LM"                               # Task type

### 2.5 Tokenizer 관련 파라미터

In [8]:
MAX_LENGTH=512                                     # Max sequence length for tokenizer
TRUNCATION=True                                     # Enable/disable truncation
RETURN_OVERFLOWING_TOKENS=True                      # Return overflowing tokens info
RETURN_LENGTH=True                                  # Return length of encoded inputs
PADDING=True                                        # Enable padding to max sequence length
PADDING_SIDE="right"                                # The side on which the model should have padding appliedㅠ

### 2.6 PROMPT 관련

In [10]:
PROMPT_DICT = {
    "prompt_input": (
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n"
        "명령어와 입력을 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.\n\n"
        
        "### Instruction(명령어):%s\n"
        "### Input(입력):%s\n"
        "### Response(응답):"
    ),
    "prompt_no_input": (
        "아래는 작업을 설명하는 명령어입니다.\n"
        "명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.\n"
        
        "### Instruction(명령어):%s\n"
        "### Response(응답):"
    ),
}

### 2.7 Evaluate 관련

* 모델 Generation 결과 데이터 직렬화 관련
    * 모델 추론 시 GPU 자원 사용 및 추론 시간이 너무 오래걸려서 결과 데이터를 직렬화

In [11]:
EVALUATE_DIR = os.path.join(BASE_PATH, 'evaluate', 'pickles')
GENERATION_DATA_LIST_PICKLE_NAME = 'sh_polyglot-alpaca_1st_1000_output.pkl'
GENERATION_DATA_LIST_PICKLE_PATH = os.path.join(EVALUATE_DIR, GENERATION_DATA_LIST_PICKLE_NAME)

* 모델 평가 결과 파일로 저장 관련 

In [12]:
EVALUATION_RESULT_DIR = os.path.join(BASE_PATH, 'evaluate', 'metrics')
EVALUATION_RESULT_FILENAME = os.path.join(EVALUATION_RESULT_DIR, 'sh_polyglot-alpaca_1th_evaluation.json')

## 3. 데이터셋 살펴보기 및 하나로 합치기

### 3.1 한국어 Aplaca 데이터셋 살펴보기

In [13]:
DATA_PATH = os.path.join(BASE_PATH, 'datas')
DATASET_KO_ALPACA_FINAL_PATH = os.path.join(DATA_PATH, 'final_ko_alpaca_data.json')

In [14]:
KO_ALPACA_FINAL_PATH = os.path.join(BASE_PATH, DATASET_KO_ALPACA_FINAL_PATH)

with open(KO_ALPACA_FINAL_PATH, 'r', encoding='utf-8') as file:
    ko_alpaca_final = json.load(file)

df_ko_alpaca_final = pd.DataFrame(ko_alpaca_final)
df_ko_alpaca_final.head(5)

,instruction,input,output
0,이메일에서 스팸을 탐지하기 위한 기계 학습 알고리즘을 만드세요.,,이메일 스팸 탐지 알고리즘을 위한 몇 가지 인기있는 기계 학습 모델에는 Suppor...
1,고기 요리 시 맛술을 사용하는 이유와 대체재는 무엇인가요? 어떤 음식에 맛술을 사용...,,안녕하세요! 고기 요리 시 맛술을 사용하는 이유는 냄새와 맛의 개선입니다. 근육 안...
2,대화를 시작하기 위해 트윗을 작성합니다.,,'안녕하세요! 오늘 날씨가 참 좋네요!' (트윗 예시)
3,저녁과 밤의 구분 기준은 무엇인가요? 저녁 8시가 맞나요? 밤 8시가 맞나요?,,저녁과 밤을 구분하는 기준은 명확히 정의되어 있지 않습니다. '저녁'은 해가 지고 ...
4,이 문장의 시작 부분에 전환 단어를 삽입하세요.,그는 오랫동안 수색한 끝에 보물을 찾았습니다.,"그러나, 그는 오랫동안 수색한 끝에 보물을 찾았습니다."


### 3.2 평가를 위해 한국어 Aplaca 데이터셋 1000개의 샘플 뽑기

In [15]:
df_ko_alpaca_final_sampled = df_ko_alpaca_final.sample(1000, random_state=RANDOM_SEED)
df_ko_alpaca_final_sampled

,instruction,input,output
17666,고객 서비스 봇에 가장 적합한 어조를 파악하세요.,,고객 서비스에서 호칭과 제품 또는 서비스에 대한 지식을 바탕으로 열정적인 어조를 사...
69026,동물 멸종 사건에 대한 뉴스 기사의 헤드라인을 생성합니다.,,동물학자들이 말하는 대규모 멸종 사건이 상상이상으로 빨리 벌어지고 있습니다.
20869,예산 수립을 위한 계획 만들기,,"예산 수립 계획을 만들 때는 다음과 같은 단계를 따릅니다. 예산 오브젝티브 설정, ..."
68447,"""Dear John""이라는 표현의 어원은 무엇일까요? 왜 이별 편지나 절교장을 의미...",,"""Dear John letter""는 이별 편지나 절교장을 뜻하는 표현으로 사용됩니다..."
15376,피보나치 수열의 처음 10개의 숫자를 출력하는 함수를 Java로 작성하십시오.,,```java\npublic class Fibonacci {\n\n public ...
...,...,...,...
7944,지구 온난화로 인해 발생할 수 있는 다섯 가지 문제를 브레인스토밍하십시오.,,"지구온난화로 인해 발생할 수 있는 다섯 가지 문제는 먹을 것 부족, 극심한 기후변화..."
1741,"""별이 태어났다""라는 문구를 다른 두 가지 방법으로 다시 쓰십시오.",,"New star is born, A star has arisen"
61640,"다음 기사의 헤드라인을 작성합니다: ""스타틴이 심장 마비 위험을 줄인다는 연구 결과...",,"""연구 결과, 스타틴은 심장 마비 위험을 줄인다."""
43819,해변의 자연물을 사용하여 게임을 디자인하세요.,,"해변에서 찾은 조개껍질을 이용하여, 조개찾기 대회를 개최해보세요."


### 3.2 평가를 위해 한국어 Aplaca 데이터셋 1000개의 샘플 뽑기

In [16]:
formatted_prompts = []
for _, row in df_ko_alpaca_final_sampled.iterrows():
    instruction = row['instruction']
    input_text = row['input']
    if input_text:
        formatted_prompts.append(PROMPT_DICT['prompt_input'] % (instruction, input_text))
    else:
        formatted_prompts.append(PROMPT_DICT['prompt_no_input'] % instruction)

In [17]:
formatted_prompts[:2]

['아래는 작업을 설명하는 명령어입니다.\n명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.\n### Instruction(명령어):고객 서비스 봇에 가장 적합한 어조를 파악하세요.\n### Response(응답):',
 '아래는 작업을 설명하는 명령어입니다.\n명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.\n### Instruction(명령어):동물 멸종 사건에 대한 뉴스 기사의 헤드라인을 생성합니다.\n### Response(응답):']

## 4. 모델 호출

### 4.1 모델 학습을 위한 기본 확인사항 내용

* 모델 파라미터 정보 확인

In [18]:
def print_trainable_parameters(model):
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    all_params = sum(p.numel() for p in model.parameters())
    print(
        f"trainable params: {trainable_params} || all params: {all_params} || trainable%: {100 * trainable_params / all_params}"
    )

* GPU 분산학습 설정 사용 유무 점검

In [19]:
ddp = False

def get_device_map():
    print(f"num_gpus: {torch.cuda.device_count()}")
    world_size = int(os.environ.get("WORLD_SIZE", torch.cuda.device_count()))
    print(f"world_size: {world_size}")
    ddp = world_size != 1
    if ddp:
        device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
        GRADIENT_ACCUMULATION_STEPS = TRAIN_BATCH_SIZE // world_size
        if GRADIENT_ACCUMULATION_STEPS == 0:
            GRADIENT_ACCUMULATION_STEPS = 1
        print(f"ddp is on - gradient_accumulation_steps: {GRADIENT_ACCUMULATION_STEPS}")
    else:
        device_map = "auto"
        print("ddp is off")

    return device_map

## 4.2 모델 Load

* 양자화 설정

In [20]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=LOAD_IN_4BIT,
    bnb_4bit_use_double_quant=BNB_4BIT_USE_DOUBLE_QUANT,
    bnb_4xqbit_quant_type=BNB_4BIT_QUANT_TYPE,
    bnb_4bit_computxe_dtype=BNB_4BIT_COMPUTE_DTYPE
)

* 모델 읽어오기

In [21]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME_OR_PATH,
    quantization_config=bnb_config,
    device_map=get_device_map(),
    cache_dir=CACHE_DIR,
    token=HUGGINGFACE_TOKEN
)

print_trainable_parameters(model)

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

num_gpus: 1
world_size: 1
ddp is off


config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/52.5k [00:00<?, ?B/s]

model-00001-of-00028.safetensors:   0%|          | 0.00/946M [00:00<?, ?B/s]

model-00002-of-00028.safetensors:   0%|          | 0.00/843M [00:00<?, ?B/s]

...

model-00027-of-00028.safetensors:   0%|          | 0.00/896M [00:00<?, ?B/s]

model-00028-of-00028.safetensors:   0%|          | 0.00/518M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/100 [00:00<?, ?B/s]

trainable params: 308848640 || all params: 6602147840 || trainable%: 4.67800248471867


## 4.3 LoRA Adapter 적용

In [22]:
model = PeftModel.from_pretrained(
    model,
    CHECKPOINT_DIR,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
)
model = model.merge_and_unload()
print_trainable_parameters(model)


if not ddp and torch.cuda.device_count() > 1:
    model.is_parallelizable = True
    model.model_parallel = True
    print("not ddp - trying its own DataParallelism")

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


trainable params: 0 || all params: 6602147840 || trainable%: 0.0


In [23]:
model.eval()

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30080, 5120)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-39): 40 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear4bit(in_features=5120, out_features=15360, bias=True)
          (dense): Linear4bit(in_features=5120, out_features=5120, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear4bit(in_features=5120, out_features=20480, bias=True)
          (dense_4h_to_h): Linear4bit(in_features=20480, out_features=

### 4.3 Tokenizer 설정

* 모델 tokenizer를 통해 사용할 데이터셋을 분석하여 입력 데이터의 길이 분포를 파악

In [24]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH, cache_dir=CACHE_DIR, use_add_token=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 5. 모델을 통한 추론

### 5.1 추론

In [25]:
streamer = TextStreamer(tokenizer)

generation_config = GenerationConfig(
    temperature=0.2,
    top_p=0.9,
    top_k=50,
    max_new_tokens=MAX_LENGTH,
    early_stopping=True,
    do_sample=True,
    repetition_penalty=1.1
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:430: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [26]:
streamer = TextStreamer(tokenizer)

def gen(instruction, input=None):
    # query = f"### instruction: {x}\n\n### Response: "
    query = PROMPT_DICT['prompt_no_input'] % instruction
    if input:
        query = PROMPT_DICT['prompt_input'] % (instruction, input)
    generated_id = model.generate(
        **tokenizer(
            query,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    generated_text = tokenizer.decode(generated_id[0], skip_special_tokens=True)

    response_start_idx = generated_text.find("### Response(응답):") + len("### Response(응답):")
    response = generated_text[response_start_idx:].strip()

    return response

## 일반 상식 예제

In [28]:
gen("""
파이썬 공부를 하고 싶습니다. 파이썬을 처음 공부해봅니다. 어떻게 공부를 시작하면 좋을까요?
""")

아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):
파이썬 공부를 하고 싶습니다. 파이썬을 처음 공부해봅니다. 어떻게 공부를 시작하면 좋을까요?

### Response(응답): 파이썬은 문법이 간단하여 쉽게 배울 수 있고, 다양한 분야에서 활용되기 때문에 많은 사람들이 배우려고 합니다. 따라서, 파이썬을 처음 공부할 때는 쉬운 책이나 인터넷 강의 등을 통해 기초부터 차근차근 배워나가면 됩니다. 또한, 파이썬 커뮤니티 사이트나 유튜브 채널 등에서도 파이썬 학습 자료를 제공하고 있으니 참고하시면 좋겠습니다.<|endoftext|>


'파이썬은 문법이 간단하여 쉽게 배울 수 있고, 다양한 분야에서 활용되기 때문에 많은 사람들이 배우려고 합니다. 따라서, 파이썬을 처음 공부할 때는 쉬운 책이나 인터넷 강의 등을 통해 기초부터 차근차근 배워나가면 됩니다. 또한, 파이썬 커뮤니티 사이트나 유튜브 채널 등에서도 파이썬 학습 자료를 제공하고 있으니 참고하시면 좋겠습니다.'

In [29]:
gen(
    instruction='컴퓨터 공학과의 일반적인 커리큘럼은 어떻게 되나요?',
    input='답변을 할 때, 2개의 문장으로 작성해주세요.'
)

아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.
명령어와 입력을 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.

### Instruction(명령어):컴퓨터 공학과의 일반적인 커리큘럼은 어떻게 되나요?
### Input(입력):답변을 할 때, 2개의 문장으로 작성해주세요.
### Response(응답): 컴퓨터 공학에서는 프로그래밍 언어, 운영체제, 데이터베이스, 네트워크 등 다양한 분야를 배웁니다.<|endoftext|>


'컴퓨터 공학에서는 프로그래밍 언어, 운영체제, 데이터베이스, 네트워크 등 다양한 분야를 배웁니다.'

## 단순 코드 계산 예제
* 코드에 대한 이해는 llama2 모델에 비해 부족해 보이며 코드에 대한 학습이 추가적으로 필요해 보임

In [30]:
gen("""
파이썬을 통해 1부터 10까지의 총합을 구하는 코드를 작성해주세요
""")

아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):
파이썬을 통해 1부터 10까지의 총합을 구하는 코드를 작성해주세요

### Response(응답): 
```python
def sum_of_numbers (num1, num2):
    for i in range(1, 11):
    if num1 == 0:
        print("종료합니다.")
    elif num2 == 0:
        print("종료합니다.")
    elif num1 == num2:
        print("두 수를 더하면 20입니다.")
    return sum_of_numbers
```<|endoftext|>


'```python\ndef sum_of_numbers (num1, num2):\n    for i in range(1, 11):\n    if num1 == 0:\n        print("종료합니다.")\n    elif num2 == 0:\n        print("종료합니다.")\n    elif num1 == num2:\n        print("두 수를 더하면 20입니다.")\n    return sum_of_numbers\n```'

In [31]:
gen("""
자바 언어를 통해 1부터 15까지 곱셈하는 코드를 작성해주세요
""")

아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):
자바 언어를 통해 1부터 15까지 곱셈하는 코드를 작성해주세요

### Response(응답): 
```java.util.Scanner;
float a, b;
int calculateFloor(float a, float b);
size = a * b;
if (size >= 15) {
    calculateFloor(size->b, size+=2);
} else if (size >= 10) {
    calculateFloor(size->b, size+=1);
} else if (size >= 7) {
    calculateFloor(size->b, size*=7);
} else {
    calculateFloor(size->b, size/=15);
} }
}
```<|endoftext|>


'```java.util.Scanner;\nfloat a, b;\nint calculateFloor(float a, float b);\nsize = a * b;\nif (size >= 15) {\n    calculateFloor(size->b, size+=2);\n} else if (size >= 10) {\n    calculateFloor(size->b, size+=1);\n} else if (size >= 7) {\n    calculateFloor(size->b, size*=7);\n} else {\n    calculateFloor(size->b, size/=15);\n} }\n}\n```'

## 요약 예제

In [32]:
gen("""
아래와 같은 기사가 있습니다. 기사의 핵심 내용을 추려서 간단하게 요약해주세요.
""",
"""
토트넘(잉글랜드)의 3연승을 이끈 '캡틴' 손흥민(31)이 맨 오브 더 매치(MOM)에 선정됐다. 
손흥민은 크리스마스 이브인 24일(한국 시각) 영국 런던의 토트넘 홋스퍼 스타디움에서 열린 에버턴과 2023-2024시즌 잉글랜드 프리미어리그(EPL) 18라운드 홈 경기에서 팀의 두 번째 골을 터뜨렸다. 
토트넘은 손흥민의 득점에 힘입어 2 대 1 승리를 거뒀다. 
이로써 토트넘은 3연승 행진을 이어갔고, 11승 3무 4패 승점 36을 기록했다. 한 경기를 덜 치른 맨체스터 시티(승점 34)를 제치고 4위로 올라섰다.
손흥민은 이날도 왼쪽 측면 공격수로 나섰다. 지난 16라운드 뉴캐슬전(1골 2도움)부터 왼쪽 측면에서 최고 윙어의 면모를 유감없이 발휘하고 있다. 
17라운드 노팅엄 포레스트전에서는 공격 포인트를 올리지 못했으나, 이날 2경기 만에 다시 득점포를 가동했다.
리그 11호 골을 터뜨린 손흥민은 무함마드 살라흐(리버풀), 재러드 보웬(웨스트햄)과 나란히 득점 공동 3위에 올랐다. 
1위는 14골의 엘링 홀란(맨체스터 시티), 2위는 12골의 도미닉 솔란케(본머스)다.
또 손흥민은 리그 반환점을 1경기 남겨둔 시점에서 벌써 지난 시즌 득점 기록을 넘어섰다. 
스포츠 탈장 부상 여파로 고전했던 지난 시즌에는 10골 6도움을 기록했다. 
도움 4개를 기록 중인 그는 지난 시즌 공격 포인트 기록 돌파도 눈앞에 두고 있다.
EPL 통산 득점 랭킹에서는 아스널의 레전드 이안 라이트(113골)을 넘어섰다. 
114골로 단독 23위에 오른 손흥민은 120골로 공동 21위인 라힘 스털링(첼시), 스티븐 제라드를 6골 차로 쫓고 있다.
손흥민은 경기 후 EPL 사무국이 22947명의 팬을 상대로 진행한 투표에서 67.7%의 압도적인 지지를 받아 MOM에 오르는 영예를 안았다. 
팀 동료인 굴리엘모 비카리오(15.1%), 페드로 포로(7.8%) 등을 크게 따돌렸다.
풀타임을 뛴 손흥민은 1골을 포함해 슈팅 2회, 패스 성공률 71%(24/34), 기회 창출 1회, 볼 터치 56회, 드리블 성공 43%(3/7) 등을 기록했다. 
축구 통계 매체 '풋몹'은 손흥민에게 팀 내 4번째로 높은 평점 7.8을 부여했다.
""")

아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.
명령어와 입력을 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.

### Instruction(명령어):
아래와 같은 기사가 있습니다. 기사의 핵심 내용을 추려서 간단하게 요약해주세요.

### Input(입력):
토트넘(잉글랜드)의 3연승을 이끈 '캡틴' 손흥민(31)이 맨 오브 더 매치(MOM)에 선정됐다. 
손흥민은 크리스마스 이브인 24일(한국 시각) 영국 런던의 토트넘 홋스퍼 스타디움에서 열린 에버턴과 2023-2024시즌 잉글랜드 프리미어리그(EPL) 18라운드 홈 경기에서 팀의 두 번째 골을 터뜨렸다. 
토트넘은 손흥민의 득점에 힘입어 2 대 1 승리를 거뒀다. 
이로써 토트넘은 3연승 행진을 이어갔고, 11승 3무 4패 승점 36을 기록했다. 한 경기를 덜 치른 맨체스터 시티(승점 34)를 제치고 4위로 올라섰다.
손흥민은 이날도 왼쪽 측면 공격수로 나섰다. 지난 16라운드 뉴캐슬전(1골 2도움)부터 왼쪽 측면에서 최고 윙어의 면모를 유감없이 발휘하고 있다. 
17라운드 노팅엄 포레스트전에서는 공격 포인트를 올리지 못했으나, 이날 2경기 만에 다시 득점포를 가동했다.
리그 11호 골을 터뜨린 손흥민은 무함마드 살라흐(리버풀), 재러드 보웬(웨스트햄)과 나란히 득점 공동 3위에 올랐다. 
1위는 14골의 엘링 홀란(맨체스터 시티), 2위는 12골의 도미닉 솔란케(본머스)다.
또 손흥민은 리그 반환점을 1경기 남겨둔 시점에서 벌써 지난 시즌 득점 기록을 넘어섰다. 
스포츠 탈장 부상 여파로 고전했던 지난 시즌에는 10골 6도움을 기록했다. 
도움 4개를 기록 중인 그는 지난 시즌 공격 포인트 기록 돌파도 눈앞에 두고 있다.
EPL 통산 득점 랭킹에서는 아스널의 레전드 이안 라이트(113골)을 넘어섰다. 
114골로 단독 23위에 오른 손흥민은 120골로 공동 21위인 라힘 스털링(첼시), 스티븐 제라드를 6골 차로 쫓고 있다.
손흥민은 경기 후 EPL 사무국이

'손흥민은 토트넘의 3연승을 이끌며 맨 오브 더 매치에 선정되었으며, EPL에서 득점 공동 3위에 올랐습니다.'

In [34]:
gen(
    instruction='아래와 같은 문장들이 있습니다. 답변을 할때 이 문장들을 2개의 문장으로 요약해주세요. 문장이 2개라는 것은 개행문자가 2개인 것을 의미합니다.',
    input=
"""
FLAN (Fine-tuned LAnguage Net) 모델은 자연어 처리(NLP) 과제들을 해결하기 위해 'instruction tuning'이라는 기법을 사용하는 언어 모델입니다.
이 모델의 핵심 아이디어는 다양한 NLP 과제를 자연어 지시사항 형태로 변형하여 이러한 과제들을 풀도록 fine-tuning하는 것입니다.
이를 통해 FLAN 모델은 번역, 상식 추론, 감정 분류 등을 포함한 다양한 NLP 과제를 수행할 수 있도록 fine-tuning 됩니다​.

FLAN의 연구 결과에 따르면, 이 모델은 zero-shot 시나리오에서 GPT-3보다 우수한 결과를 보였으며, 많은 task에서는 supervised model과 비슷한 성능을 달성했습니다.
특히 자연어 추론(NLI)과 질문응답(QA) 작업에서 효과적이었습니다.
Google Research Blog에서는 FLAN이 언어 모델을 사용하여 특정 실제 과제에 대한 지식을 어떻게 풀어내는지에 대해 설명합니다.
전통적으로는 레이블이 붙은 데이터셋을 이용해 fine-tuning하는 방법이 많이 사용되었지만, FLAN은 다양한 종류의 지시사항에 대해 모델을 fine-tuning함으로써, 특정 과제가 아닌 일반적인 NLP 과제들을 해결할 수 있게 만듭니다.
""")

아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.
명령어와 입력을 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.

### Instruction(명령어):아래와 같은 문장들이 있습니다. 답변을 할때 이 문장들을 2개의 문장으로 요약해주세요. 문장이 2개라는 것은 개행문자가 2개인 것을 의미합니다.
### Input(입력):
FLAN (Fine-tuned LAnguage Net) 모델은 자연어 처리(NLP) 과제들을 해결하기 위해 'instruction tuning'이라는 기법을 사용하는 언어 모델입니다.
이 모델의 핵심 아이디어는 다양한 NLP 과제를 자연어 지시사항 형태로 변형하여 이러한 과제들을 풀도록 fine-tuning하는 것입니다.
이를 통해 FLAN 모델은 번역, 상식 추론, 감정 분류 등을 포함한 다양한 NLP 과제를 수행할 수 있도록 fine-tuning 됩니다​.

FLAN의 연구 결과에 따르면, 이 모델은 zero-shot 시나리오에서 GPT-3보다 우수한 결과를 보였으며, 많은 task에서는 supervised model과 비슷한 성능을 달성했습니다.
특히 자연어 추론(NLI)과 질문응답(QA) 작업에서 효과적이었습니다.
Google Research Blog에서는 FLAN이 언어 모델을 사용하여 특정 실제 과제에 대한 지식을 어떻게 풀어내는지에 대해 설명합니다.
전통적으로는 레이블이 붙은 데이터셋을 이용해 fine-tuning하는 방법이 많이 사용되었지만, FLAN은 다양한 종류의 지시사항에 대해 모델을 fine-tuning함으로써, 특정 과제가 아닌 일반적인 NLP 과제들을 해결할 수 있게 만듭니다.

### Response(응답): FLAN 모델은 자연어 처리(NLP) 과제를 해결하기 위해 'instruction tuning'이라는 기법을 사용하는 언어 모델입니다. FLAN 모델은 다양한 NLP 과제를 해결할 수 있도록 fine-tuning하며, 이를 통해 자연어 추론(NLI) 및 질문응답(QA)

"FLAN 모델은 자연어 처리(NLP) 과제를 해결하기 위해 'instruction tuning'이라는 기법을 사용하는 언어 모델입니다. FLAN 모델은 다양한 NLP 과제를 해결할 수 있도록 fine-tuning하며, 이를 통해 자연어 추론(NLI) 및 질문응답(QA) 작업에서 높은 성과를 보입니다."

## One-shot 예제

In [40]:
gen("""
Blending is all you need 가 뭐에요?
""")

아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):
Blending is all you need 가 뭐에요?

### Response(응답): Blending은 혼합이라는 뜻으로, 이미지의 색상과 톤을 자동으로 조정하여 자연스러운 이미지를 만들어내는 기능입니다. 이 명령어는 이미지 편집 프로그램에서 사용할 수 있습니다.<|endoftext|>


'Blending은 혼합이라는 뜻으로, 이미지의 색상과 톤을 자동으로 조정하여 자연스러운 이미지를 만들어내는 기능입니다. 이 명령어는 이미지 편집 프로그램에서 사용할 수 있습니다.'

In [41]:
gen("""
질문 : Blending is all you need 가 뭐에요? 이 논문에 대한 abstract의 내용을 하단의 External Generated Knowledge로 드립니다.

```
대화형 AI 연구에서는 ChatGPT와 같은 모델들을 예로 들며 매개변수가 많은 모델을 개발하는 경향이 눈에 띕니다. 
이러한 광범위한 모델들은 점점 더 나은 채팅 응답을 생성하는 경향이 있지만, 상당한 계산 자원과 메모리를 요구합니다. 
이 연구는 중요한 질문을 탐구합니다: 
작은 모델들의 조합이 협력적으로 단일 큰 모델에 비해 비슷하거나 향상된 성능을 달성할 수 있는가? 
우리는 "블렌딩"이라는 명칭의, 간단하지만 효과적인 여러 대화형 AI 모델들을 통합하는 방법을 소개합니다. 
실증적 증거는 특정 작은 모델들이 시너지적으로 혼합될 때, 훨씬 더 큰 모델들의 능력을 능가하거나 맞추어 줄 수 있다고 제안합니다. 
예를 들어, 중간 크기의 모델 세 개(6B/13B 매개변수)만 통합해도 ChatGPT(175B+ 매개변수)와 같은 훨씬 큰 모델의 성능 지표를 능가하거나 맞출 수 있습니다.
이 가설은 Chai 연구 플랫폼에서 대규모 사용자 기반을 대상으로 하는 A/B 테스팅 방법론을 사용하여 30일 동안 엄격하게 테스트되었습니다. 
연구 결과는 컴퓨팅 요구 사항이 급증하지 않는 상황에서 대화형 AI 효율성을 향상시킬 수 있는 "블렌딩" 전략의 잠재력을 강조합니다.
```
""")

아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):
질문 : Blending is all you need 가 뭐에요? 이 논문에 대한 abstract의 내용을 하단의 External Generated Knowledge로 드립니다.

```
대화형 AI 연구에서는 ChatGPT와 같은 모델들을 예로 들며 매개변수가 많은 모델을 개발하는 경향이 눈에 띕니다. 
이러한 광범위한 모델들은 점점 더 나은 채팅 응답을 생성하는 경향이 있지만, 상당한 계산 자원과 메모리를 요구합니다. 
이 연구는 중요한 질문을 탐구합니다: 
작은 모델들의 조합이 협력적으로 단일 큰 모델에 비해 비슷하거나 향상된 성능을 달성할 수 있는가? 
우리는 "블렌딩"이라는 명칭의, 간단하지만 효과적인 여러 대화형 AI 모델들을 통합하는 방법을 소개합니다. 
실증적 증거는 특정 작은 모델들이 시너지적으로 혼합될 때, 훨씬 더 큰 모델들의 능력을 능가하거나 맞추어 줄 수 있다고 제안합니다. 
예를 들어, 중간 크기의 모델 세 개(6B/13B 매개변수)만 통합해도 ChatGPT(175B+ 매개변수)와 같은 훨씬 큰 모델의 성능 지표를 능가하거나 맞출 수 있습니다.
이 가설은 Chai 연구 플랫폼에서 대규모 사용자 기반을 대상으로 하는 A/B 테스팅 방법론을 사용하여 30일 동안 엄격하게 테스트되었습니다. 
연구 결과는 컴퓨팅 요구 사항이 급증하지 않는 상황에서 대화형 AI 효율성을 향상시킬 수 있는 "블렌딩" 전략의 잠재력을 강조합니다.
```

### Response(응답): "Blending is all you need"는 대화형 AI 모델을 통합하는 방법 중 하나입니다. 작은 모델들을 통합하면 더 큰 모델보다 더 나은 성능을 발휘할 수 있다는 것을 보여줍니다.<|endoftext|>


'"Blending is all you need"는 대화형 AI 모델을 통합하는 방법 중 하나입니다. 작은 모델들을 통합하면 더 큰 모델보다 더 나은 성능을 발휘할 수 있다는 것을 보여줍니다.'

## CoT

* 수학적 연산을 LLM이 제대로 하지 못함

In [42]:
gen("""
한 반에 30명의 학생이 있습니다. 그 중 3분의 2가 소녀입니다. 소년은 몇 명입니까?
""")

아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):
한 반에 30명의 학생이 있습니다. 그 중 3분의 2가 소녀입니다. 소년은 몇 명입니까?

### Response(응답): 13명<|endoftext|>


'13명'

* CoT의 Few-shot 예제를 추가하였음에도 제대로 답변을 못함
    * KoAlpaca 데이터만 학습을 하였기에 LLM이 수학적인 사고를 하지 못함
* 다음번에 학습시킬 때에는 CoT 데이터셋을 추가하고 실험을 해서 CoT 문제를 해결해보자!

In [43]:
gen(
    instruction = '한 반에는 소년과 소녀로 이루어져 있으며, 총 60명의 학생이 있습니다. 그 중 3분의 2가 소녀입니다. 소년은 몇 명입니까? input의 예제를 보고 논리적으로 분해하고 생각을 하여 올바른 답변을 해주세요',
    input=
"""
예제1)
먼저, 반 전체 학생 수인 24명 중에서 소녀의 비율을 계산해야 합니다. 소녀는 전체의 4분의 3에 해당합니다.
24명을 4로 나누면 각 그룹에 몇 명이 있는지 알 수 있습니다. 24를 4로 나누면 6명입니다.
이제 이 수를 3배하면 소녀의 수를 알 수 있습니다. 6명의 3배는 18명입니다. 그러므로 소녀는 18명입니다.
전체 학생 수에서 소녀의 수를 빼면 소년의 수를 알 수 있습니다. 24명에서 18명을 빼면 6명이 남습니다.
따라서 이 반에는 소년이 6명 있습니다.
Response: 6명

예제2)
먼저, 반 전체 학생 수인 28명 중에서 소녀의 비율을 계산해야 합니다. 소녀는 전체의 3분의 2에 해당합니다.
28명을 3으로 나누면 각 그룹에 몇 명이 있는지 알 수 있습니다. 28을 3으로 나누면 9.33, 즉 약 9명입니다.
이제 이 수를 2배하면 소녀의 수를 알 수 있습니다. 9명의 2배는 18명입니다. 그러므로 소녀는 18명입니다.
전체 학생 수에서 소녀의 수를 빼면 소년의 수를 알 수 있습니다. 28명에서 18명을 빼면 10명이 남습니다.
따라서 이 반에는 소년이 10명 있습니다.
Response: 10명

예제3)
먼저, 도서관 전체 책의 수인 30권 중에서 과학 책의 비율을 계산해야 합니다. 과학 책은 전체의 5분의 4에 해당합니다.
30권을 5로 나누면 각 그룹에 몇 권이 있는지 알 수 있습니다. 30을 5로 나누면 6권입니다.
이제 이 수를 4배하면 과학 책의 수를 알 수 있습니다. 6권의 4배는 24권입니다. 그러므로 과학 책은 24권입니다.
전체 책의 수에서 과학 책의 수를 빼면 문학 책의 수를 알 수 있습니다. 30권에서 24권을 빼면 6권이 남습니다.
따라서 도서관에는 문학 책이 6권 있습니다.
Response : 6권

그렇다면 instruction의 정답은??
""")

아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.
명령어와 입력을 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.

### Instruction(명령어):한 반에는 소년과 소녀로 이루어져 있으며, 총 60명의 학생이 있습니다. 그 중 3분의 2가 소녀입니다. 소년은 몇 명입니까? input의 예제를 보고 논리적으로 분해하고 생각을 하여 올바른 답변을 해주세요
### Input(입력):
예제1)
먼저, 반 전체 학생 수인 24명 중에서 소녀의 비율을 계산해야 합니다. 소녀는 전체의 4분의 3에 해당합니다.
24명을 4로 나누면 각 그룹에 몇 명이 있는지 알 수 있습니다. 24를 4로 나누면 6명입니다.
이제 이 수를 3배하면 소녀의 수를 알 수 있습니다. 6명의 3배는 18명입니다. 그러므로 소녀는 18명입니다.
전체 학생 수에서 소녀의 수를 빼면 소년의 수를 알 수 있습니다. 24명에서 18명을 빼면 6명이 남습니다.
따라서 이 반에는 소년이 6명 있습니다.
Response: 6명

예제2)
먼저, 반 전체 학생 수인 28명 중에서 소녀의 비율을 계산해야 합니다. 소녀는 전체의 3분의 2에 해당합니다.
28명을 3으로 나누면 각 그룹에 몇 명이 있는지 알 수 있습니다. 28을 3으로 나누면 9.33, 즉 약 9명입니다.
이제 이 수를 2배하면 소녀의 수를 알 수 있습니다. 9명의 2배는 18명입니다. 그러므로 소녀는 18명입니다.
전체 학생 수에서 소녀의 수를 빼면 소년의 수를 알 수 있습니다. 28명에서 18명을 빼면 10명이 남습니다.
따라서 이 반에는 소년이 10명 있습니다.
Response: 10명

예제3)
먼저, 도서관 전체 책의 수인 30권 중에서 과학 책의 비율을 계산해야 합니다. 과학 책은 전체의 5분의 4에 해당합니다.
30권을 5로 나누면 각 그룹에 몇 권이 있는지 알 수 있습니다. 30을 5로 나누면 6권입니다.
이제 이 수를 4배하면 과학 책의 수를 알 수 있습니다. 6권의 4배는 24권입니다. 

'소년은 12명, 소녀는 8명입니다.'

## 테스트 데이터셋 검증(held-out 데이터는 아님)¶

In [44]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_k=1,
    num_beams=3,
    do_sample=True,
)

In [48]:
generated_texts = []

for prompt in formatted_prompts[:10]:
    model_input = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LENGTH).to('cuda:0')
    model_input = {key: val.to('cuda:0') for key, val in model_input.items() if key != 'token_type_ids'}

    generated_id = model.generate(
        **model_input,
        max_new_tokens=MAX_LENGTH,
        early_stopping=True,
        generation_config=generation_config,
        output_scores=True,
        return_dict_in_generate=False,
    )

    generated_text = tokenizer.decode(generated_id[0], skip_special_tokens=True)
    print(generated_text)
    print('-' * 100)
    generated_texts.append(generated_text)

final_responses = []
for output in generated_texts:
    response_start_idx = output.find("### Response(응답):") + len("### Response(응답):")
    response = output[response_start_idx:].strip()
    final_responses.append(response)

final_responses

아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):고객 서비스 봇에 가장 적합한 어조를 파악하세요.
### Response(응답): 친절하고 부드러운 어조가 고객 서비스 봇에 가장 적합합니다.
----------------------------------------------------------------------------------------------------
아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):동물 멸종 사건에 대한 뉴스 기사의 헤드라인을 생성합니다.
### Response(응답):"동물 멸종, 지구상에서 가장 큰 위기 직면"
----------------------------------------------------------------------------------------------------
아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):예산 수립을 위한 계획 만들기
### Response(응답):예산 수립을 위한 계획을 만들기 위해서는 다음과 같은 단계를 따라야 합니다. 1) 목표를 설정하고 2) 예산을 수립하고 3) 예산을 집행합니다.
----------------------------------------------------------------------------------------------------
아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):"Dear John"이라는 표현의 어원은 무엇일까요? 왜 이별 편지나 절교장을 의미하는 표현으로 사용되는 것일까요?

영어 사전에서 "Dear"를 검색하다 보면 여러 가지 뜻이 나오는데, 그 중에 하나가 "Dear Jo

['친절하고 부드러운 어조가 고객 서비스 봇에 가장 적합합니다.',
 '"동물 멸종, 지구상에서 가장 큰 위기 직면"',
 '예산 수립을 위한 계획을 만들기 위해서는 다음과 같은 단계를 따라야 합니다. 1) 목표를 설정하고 2) 예산을 수립하고 3) 예산을 집행합니다.',
 '"Dear John"은 이별 편지나 절교장을 의미하는 표현으로 사용됩니다. 이 용어는 "존에게"라는 뜻으로, "존"은 영어로 "John"이라고 쓰입니다. "Dear John"은 "존에게"라는 의미를 가지고 있습니다. 이 용어는 이별 편지나 절교장을 의미하는 것으로 사용됩니다. 이 용어의 어원은 정확하게 알려진 바는 없지만, "존"이라는 이름이 "John"으로 쓰이는 것과 관련이 있을 것으로 추측됩니다. 또한, "Dear John"은 "존에게"라는 의미를 가지고 있기 때문에, "존에게"라는 의미를 가진 다른 표현들과도 연관이 있을 것으로 추측됩니다.',
 'int sum(int a, int b) { return a + b * b; }',
 '비밀번호를 사용하고, 안전한 클라우드 서비스를 사용하며, 데이터를 암호화하고, 보안 소프트웨어를 사용하며, 정기적인 백업을 수행하는 것이 좋습니다.',
 '1. XYZ 주식의 가격이 상승할 때, 2. XYZ 주식의 가격이 하락할 때, 3. XYZ 주식의 가격이 일정한 범위 내에서 유지될 때, 4. XYZ 주식의 가격이 상승할 때, 5. XYZ 주식의 가격이 하락할 때.',
 '원격조정은 멀리 떨어진 곳에서 컴퓨터를 제어하는 것을 말합니다. 인터넷을 통해 멀리 떨어진 곳에서도 컴퓨터를 제어할 수 있습니다. 원격조정은 주로 네트워크 보안, 원격지원, 원격제어 등의 용도로 사용됩니다. 원격조정은 주로 인터넷을 통해 이루어지기 때문에, 인터넷이 연결되어 있어야 합니다. 원격조정은 주로 컴퓨터를 제어하는 데 사용됩니다. 원격조정은 멀리 떨어진 곳에서도 컴퓨터를 제어할 수 있는 편리한 방법입니다.',
 '마을에 도착한 후, 마을의 비밀을 파헤치는 모험을 시작하게 

### 5.2 추론한 내용 다음 번에 재활용하기 위해 pickle 파일로 직렬화

In [ ]:
import pickle
with open(GENERATION_DATA_LIST_PICKLE_PATH, 'wb') as file:
    pickle.dump(final_responses, file)

### 5.3 저장되어져있는 추론 내용 pickle 파일로 역직렬화를 통한 메모리 로드

In [ ]:
import pickle

with open(GENERATION_DATA_LIST_PICKLE_PATH, 'rb') as file:
    generation_data_list = pickle.load(file)

## 6. 데이터 평가

In [ ]:
reference_list = df_ko_alpaca_final_sampled['output'].tolist()
print(len(reference_list))
print(reference_list[0])

In [ ]:
from evaluate import load
import json

class LLMEvaluator:
    pass

In [ ]:
evaluator = LLMEvaluator()
result = evaluator.run_evaluate(
    predictions=generation_data_list, 
    origins=reference_list,
    use_save_as_file=True,
)